In [ ]:
!pip install python-kucoin

In [51]:
from kucoin.client import Client
import requests
import datetime 
import pandas as pd
import numpy as np
import time 

api_key = ''
api_secret = ''
api_passphrase = ''

CCData_api_key = ''

client = Client(api_key, api_secret, api_passphrase)

def buy_order(size):
    order = client.create_market_order('BTC-USDT', Client.SIDE_BUY, size=size)
    
def sell_order(size):
    order = client.create_market_order('BTC-USDT', Client.SIDE_BUY, size=size)

def mAlphaTrend(df, inpPeriods=14, inpMultiplier=1.618, inpATRSmoothType='Exponential', inpVolumeData=False):
    
    # Initialize indicator components
    atr = df['low'].rolling(window=inpPeriods).apply(lambda x: np.mean(np.abs(np.diff(x))), raw=True)
    rsi = 100 - (100 / (1 + df['close'].rolling(window=14).apply(lambda x: np.mean(np.where(np.diff(x) > 0, np.diff(x), 0)),raw=True)/
            -df['close'].rolling(window=14).apply(lambda x: np.mean(np.where(np.diff(x) < 0, np.diff(x), 0)), raw=True)))
    mfi = (df['close'] + df['high'] + df['low']) / 3 * df['volumeto']
    mfi = 100- (100 / (1 + mfi.rolling(window=14).apply(lambda x: np.mean(np.where(np.diff(x) > 0, np.diff(x), 0)),raw=True)/
            -mfi.rolling(window=14).apply(lambda x: np.mean(np.where(np.diff(x) < 0, np.diff(x), 0)), raw=True)))
    
    # Initialize result columns
    trendup = df['low'] - atr * inpMultiplier
    trenddn = df['high'] + atr * inpMultiplier
    alphatrend = np.nan
    trend = np.nan
    previous_alphatrend = np.nan
    
    # Calculate indicator values
    for i in range(len(df)):
        if inpVolumeData:
            condition = (rsi[i] >= 60) & (mfi[i] >= 70)
            
        if condition:
            if trendup[i] < alphatrend:
                alphatrend = alphatrend if np.isnan(alphatrend) else trendup[i]
            else:
                alphatrend = trendup[i]
        else:
            if trenddn[i] > alphatrend:
                alphatrend = alphatrend if np.isnan(alphatrend) else trenddn[i]
            else:
                alphatrend = trenddn[i]
        
        if alphatrend > previous_alphatrend:
            trend = 1
        elif alphatrend < previous_alphatrend:
            trend = -1
        else:
            trend = trend
        
        df.at[df.index[i], 'AlphaTrend'] = alphatrend
        df.at[df.index[i], 'AlphaTrendBullish'] = np.nan if trend != 1 else alphatrend
        df.at[df.index[i], 'AlphaTrendBearish'] = np.nan if trend != -1 else alphatrend
        
        # Update previous_alphatrend for the next iteration
        previous_alphatrend = alphatrend
    
    return df[['AlphaTrend', 'AlphaTrendBullish', 'AlphaTrendBearish']]



def price_day(fsym, tsym, limit, api_key, freq='day'):
    url = (f'https://min-api.cryptocompare.com/data/v2/histo{freq}?'
           f'fsym={fsym}&tsym={tsym}&limit={limit}&e=cccagg&'
           f'tryConversion=true&api_key={api_key}')
    data = requests.get(url).json()
    df = pd.DataFrame(data['Data']['Data'])
    df.index = pd.to_datetime(df['time'], unit='s')

    return df

In [59]:
import time 

trades = []
Wallet = 100
Invested = 0 
net_profit = []

aggregation_functions = {'time': 'last', 'close': 'last', 'open' : 'first',
                         'low': 'min', 'high': 'max', 'volumeto': 'sum'}

prices = price_day('BTC', 'USDT', 2000, CCData_api_key, freq='minute')
prices = price.resample('5Min').agg(aggregation_functions)

first_iteration = True  


try:
    while True:
        price = price_day('BTC', 'USDT', 100, CCData_api_key, freq='minute')
        prices = pd.concat([prices, price.tail(1)]) 

        df = mAlphaTrend(prices, inpPeriods=14, inpMultiplier=1.618,
                         inpATRSmoothType='Exponential', inpVolumeData=True)
        if first_iteration:
            trades = [[0,df['AlphaTrend'].iloc[-1]]]
            first_iteration = False

        if (df['AlphaTrend'].iloc[-1] == df['AlphaTrendBullish'].iloc[-1]) & (Invested == 0):
            trades.append([df.index[-1], df['AlphaTrend'][-1], 'BUY'])
            net_profit.append([df.index[-1],Wallet])
            Invested = (Wallet/df['AlphaTrend'].iloc[-1])
            Wallet = 0
            print('Buy Order took Place @ price', prices.close.tail(1))

        if (df['AlphaTrend'].iloc[-1] == df['AlphaTrendBearish'].iloc[-1]) & (Invested > 0):
            trades.append([df.index[-1], df['AlphaTrend'].iloc[-1], 'SELL'])
            Wallet = Invested*df['AlphaTrend'].iloc[-1] - 0.01
            Invested = 0 
            print('Sell Order took Place @ price', prices.close.tail(1))
            print('Wallet have amount', Wallet)

        time.sleep(60) 

except KeyboardInterrupt:
    print("Tickerfetching stopped.")    

Buy Order took Place @ price time
2023-08-29 15:06:00    27472.52
Name: close, dtype: float64
Sell Order took Place @ price time
2023-08-29 15:07:00    27375.57
Name: close, dtype: float64
Wallet have amount 99.9129393627841


ConnectionError: HTTPSConnectionPool(host='min-api.cryptocompare.com', port=443): Max retries exceeded with url: /data/v2/histominute?fsym=BTC&tsym=USDT&limit=100&e=cccagg&tryConversion=true&api_key=61fd48a1d098f28f6015d7283be5477c06f040f87badec4b03f592b0c9bbaeb0 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x138c527d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))